### The following code reads the forward and trans model and creates the reconstructed beamformer source activity using LCMV filters.


need epoch data; the following script is not complete and assumes that the epoch/ evoked data is already loaded.

# # Read forward model and trans file

In [ ]:
import mne
from mne.beamformer import apply_lcmv, make_lcmv
from mne import read_source_estimate

subject = subject

base_dir = (r'C:\Users\lpxaj7\OneDrive - The University of Nottingham\Documents\Experiments\Year_1\v0_1')
fs_path = os.path.join(base_dir,'DA\\all_files', subject, '')



fwd_fname = str(fs_path + f'{subject}-fwd.fif')
forward = mne.read_forward_solution(fwd_fname)
forward['info']


#trans file

import numpy as np
from mne.transforms import apply_trans, Transform

trans_file=(fs_path + f'{subject}-trans.fif')  #'15463_2-trans.fif'

if subject =='15463':
    trans_file=(fs_path + f'{subject}_2-trans.fif')  #'15463
    
# Get the transformation matrix
mri_trans = mne.read_trans(trans_file)  # Transformation matrix

subjects_dir=fs_path


# Define Epochs for visualisation

In [ ]:
##%%%% define values of epocch/ evoked and mni cords to plot
##%% epoch visuLalization
right_ep= epochs_new["rightVF"] # right vf implies left hemisphere brain areas to look for 
left_ep = epochs_new["leftVF"]

#left epoch
epoch_visualise = left_ep

evoked = epoch_visualise.average()
evoked.plot()


#mni cord to plot (mm)
# cord=[36, -28, 37]  # right sensorimotor in mm

#right_epochs

# epoch_visualise = right_ep
# #mni cord to plot (mm)
# cord=[-36,-14,56]  # left sensorimotor

# compute active and data cov matrix
active window is window of interest and control is baseline/rest period for the epoch data

In [ ]:
##%%%%%%%%%%%%%%%%%%% sourcespace
active_data_cov=  mne.compute_covariance(epoch_visualise, method='empirical', tmin=0.02, tmax=0.2)
control_data_cov=  mne.compute_covariance(epoch_visualise, method='empirical', tmin=0.6, tmax=0.78)
data_cov=active_data_cov+control_data_cov
figs= data_cov.plot(epoch_visualise.info)

#### if the channels in forward model and data cov is not equal to channels in info object
#### then it causes a problem, therefore if needed restrict the forward solution to picked/ selected channels

In [ ]:
# Restrict forward solution to the picked/selected channels
forward_pick = forward.copy().pick_channels(ch_names=evoked.info['ch_names'])

# Restrict data covariance to the picked channels
data_cov_pick = data_cov.copy().pick_channels(ch_names=evoked.info['ch_names'])

# save a bit of memory
src = forward_pick["src"]
# del forward_pick

# make LCMV filters

In [ ]:
# right_ep.load_data()  # This loads the data into memory
# evoked_left_beta=right_ep.copy().filter(13,30).average()

filters = make_lcmv(
    evoked.info,
    forward_pick,
    data_cov_pick,
    reg=0.05,
    pick_ori="max-power",
    weight_norm="unit-noise-gain",
    rank=None,
)

# vector filters  # if needed
filters_vec = make_lcmv(
    evoked.info,
    forward_pick,
    data_cov_pick,
    reg=0.05,
    pick_ori="vector",
    weight_norm="unit-noise-gain-invariant", #unit-noise-gain 
    rank=None,
)

# compute pseudo t-stats

Calculate the changed in relative power

In [ ]:
# compute pseudoi t stats
stc_active=mne.beamformer.apply_lcmv_cov(active_data_cov, filters)
stc_control=mne.beamformer.apply_lcmv_cov(control_data_cov, filters)
stc= (stc_active-stc_control)/stc_control   ## when calculating stc active-control /pseudoo tstqats, we get only one time point value so no pattern in 2nd panel
##compares power between active and control periods

# visualise reconstructed source activity

In [ ]:
lims=[0.3, 0.45, 0.6]

kwargs= dict(
    src=src,
    subject=subject,
    subjects_dir=fs_path,
   # initial_pos=mri_voxel_coords,   # initial_pos=(36, -28, 37),
    verbose=True
    )

# visualise reconstructed activity
#ON MRI slices
brain=stc.plot(**kwargs)  ## when calculating stc active-control /pseudoo tstqats, we get only one time point value so no pattern in 2nd panel

#on glass brain
brain=stc.plot(mode="glass_brain",**kwargs)  ## when calculating stc active-control /pseudoo tstqats, we get only one time point value so no pattern in 2nd panel
